In [1]:
import pandas

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data


In [2]:
df = px.data.tips()# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )# Run app and display result inline in the notebook
app.run_server(mode='jupyterlab')
#app.run_server(mode='inline')
#app.run_server(mode='external')


In [ ]:
#Component
#plotly graps
# callbacks

In [3]:
import pandas as pd
import geopandas as gpd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [6]:
aoi = gpd.read_file('F:/binder-dashboard/dashboard/type_.shp')

DriverError: F:/binder-dashboard/dashboard/fields_type.shp: No such file or directory

In [12]:
df = gpd.read_file(
    "https://github.com/florisabrams/binder_dashboard/tree/main/dashboard/type_fields.shp",
    low_memory=False,)

DriverError: '/vsimem/84f19b36e8aa43609cb93a8ac88d3687' not recognized as a supported file format.